In [1]:
!pip install sqlalchemy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### Creating database.

In [2]:
import sqlalchemy as db


engine = db.create_engine('sqlite:///Enterprise.db')
connection = engine.connect()
metadata = db.MetaData()

### Creating Department table

In [3]:
department = db.Table('Department', metadata,
                      db.Column('id', db.Integer, primary_key=True),
                      db.Column('department_name', db.String(50), nullable=False))

### Creating Job Title table

In [4]:
job_title = db.Table('Job_Title', metadata,
                     db.Column('id', db.Integer, primary_key=True),
                     db.Column('job_name', db.String(50), nullable=False))

### Creating Employees table

In [5]:
employees = db.Table('Employees', metadata,
                     db.Column('id', db.Integer, primary_key=True),
                     db.Column('name', db.String(50), nullable=False),
                     db.Column('department_id', db.Integer, db.ForeignKey('Department.id')),
                     db.Column('job_title_id', db.Integer, db.ForeignKey('Job_Title.id')))

### Creating Salary table

In [6]:
salary = db.Table('Salary', metadata,
                  db.Column('id', db.Integer, primary_key=True),
                  db.Column('amount', db.Integer, nullable=False),
                  db.Column('employee_id', db.Integer, db.ForeignKey('Employees.id'), unique=True))

### Adding tables to the database

In [7]:
metadata.create_all(engine)

### Inserting values to Department

In [8]:
connection.execute(
    department.insert().values([
        {'department_name': 'IT'},
        {'department_name': 'Service'},
        {'department_name': 'Finance'}
]))

### Inserting values to Job Title

In [9]:
connection.execute(
    job_title.insert().values([
        {'job_name': 'Software Engineer'},
        {'job_name': 'QA Engineer'},
        {'job_name': 'Physical Security'},
        {'job_name': 'Accountant'},
        {'job_name': 'Plumber'}
]))

### Inserting values to Employee

In [10]:
connection.execute(
    employees.insert().values([
        {'name': 'John Smith', 'department_id': 1, 'job_title_id': 1},
        {'name': 'Sherlock Holmes', 'department_id': 1, 'job_title_id': 2},
        {'name': 'Stephen King', 'department_id': 2, 'job_title_id': 3},
        {'name': 'David Beckham', 'department_id': 1, 'job_title_id': 1},
        {'name': 'Mary Jane', 'department_id': 3, 'job_title_id': 4},
        {'name': 'Samanta Smith', 'department_id': 3, 'job_title_id': 4},
        {'name': 'Jason Statham', 'department_id': 2, 'job_title_id': 3},
        {'name': 'Sam Smith', 'department_id': 2, 'job_title_id': 5},
        {'name': 'Will Smith', 'department_id': 1, 'job_title_id': 2},
        {'name': 'Jon Jones', 'department_id': 2, 'job_title_id': 3},
        {'name': 'Connor McGregor', 'department_id': 2, 'job_title_id': 3},
]))

### Inserting values to Salary

In [11]:
import random


connection.execute(
    salary.insert().values([
        {'amount': random.randint(500, 10000), 'employee_id': 1},
        {'amount': random.randint(500, 10000), 'employee_id': 2},
        {'amount': random.randint(500, 10000), 'employee_id': 3},
        {'amount': random.randint(500, 10000), 'employee_id': 4},
        {'amount': random.randint(500, 10000), 'employee_id': 5},
        {'amount': random.randint(500, 10000), 'employee_id': 6},
        {'amount': random.randint(500, 10000), 'employee_id': 7},
        {'amount': random.randint(500, 10000), 'employee_id': 8},
        {'amount': random.randint(500, 10000), 'employee_id': 9},
        {'amount': random.randint(500, 10000), 'employee_id': 10},
        {'amount': random.randint(500, 10000), 'employee_id': 11},
]))

### Displaying all the content from the tables

In [12]:
connection.exec_driver_sql('SELECT '
                           '    * '
                           'FROM '
                           '    Salary AS s '
                           'JOIN '
                           '    Employees AS e ON s.employee_id = e.id '
                           'JOIN '
                           '    Department AS d ON e.department_id = d.id '
                           'JOIN '
                           '    Job_Title AS jt ON e.job_title_id = jt.id '
                           ).fetchall()

[(1, 4146, 1, 1, 'John Smith', 1, 1, 1, 'IT', 1, 'Software Engineer'),
 (2, 7698, 2, 2, 'Sherlock Holmes', 1, 2, 1, 'IT', 2, 'QA Engineer'),
 (3, 2240, 3, 3, 'Stephen King', 2, 3, 2, 'Service', 3, 'Physical Security'),
 (4, 4411, 4, 4, 'David Beckham', 1, 1, 1, 'IT', 1, 'Software Engineer'),
 (5, 599, 5, 5, 'Mary Jane', 3, 4, 3, 'Finance', 4, 'Accountant'),
 (6, 6629, 6, 6, 'Samanta Smith', 3, 4, 3, 'Finance', 4, 'Accountant'),
 (7, 4877, 7, 7, 'Jason Statham', 2, 3, 2, 'Service', 3, 'Physical Security'),
 (8, 8453, 8, 8, 'Sam Smith', 2, 5, 2, 'Service', 5, 'Plumber'),
 (9, 2200, 9, 9, 'Will Smith', 1, 2, 1, 'IT', 2, 'QA Engineer'),
 (10, 7713, 10, 10, 'Jon Jones', 2, 3, 2, 'Service', 3, 'Physical Security'),
 (11, 7644, 11, 11, 'Connor McGregor', 2, 3, 2, 'Service', 3, 'Physical Security')]

### Finding average salary by department

In [13]:
connection.exec_driver_sql('SELECT '
                           '    d.department_name, '
                           '    AVG(s.amount) AS AVG_SALARY '
                           'FROM '
                           '    Salary AS s '
                           'JOIN '
                           '    Employees AS e ON s.employee_id = e.id '
                           'JOIN '
                           '    Department AS d ON e.department_id = d.id '
                           'JOIN '
                           '    Job_Title AS jt ON e.job_title_id = jt.id '
                           'GROUP BY '
                           '    d.department_name ').fetchall()

[('Finance', 3614.0), ('IT', 4613.75), ('Service', 6185.4)]

### Finding average salary by job title

In [14]:
connection.exec_driver_sql('SELECT '
                           '    jt.job_name, '
                           '    AVG(s.amount) AS AVG_SALARY '
                           'FROM '
                           '    Salary AS s '
                           'JOIN '
                           '    Employees AS e ON s.employee_id = e.id '
                           'JOIN '
                           '    Department AS d ON e.department_id = d.id '
                           'JOIN '
                           '    Job_Title AS jt ON e.job_title_id = jt.id '
                           'GROUP BY '
                           '    jt.job_name ').fetchall()

[('Accountant', 3614.0),
 ('Physical Security', 5618.5),
 ('Plumber', 8453.0),
 ('QA Engineer', 4949.0),
 ('Software Engineer', 4278.5)]

### Finding departments with quantity of employees > 3

In [15]:
connection.exec_driver_sql('SELECT '
                           '    d.department_name, '
                           '    COUNT(e.name) AS EMPLOYEE_COUNT '
                           'FROM '
                           '    Salary AS s '
                           'JOIN '
                           '    Employees AS e ON s.employee_id = e.id '
                           'JOIN '
                           '    Department AS d ON e.department_id = d.id '
                           'JOIN '
                           '    Job_Title AS jt ON e.job_title_id = jt.id '
                           'GROUP BY '
                           '    d.department_name '
                           'HAVING '
                           '    EMPLOYEE_COUNT > 3 '
                           'ORDER BY '
                           '    EMPLOYEE_COUNT DESC ').fetchall()

[('Service', 5), ('IT', 4)]

### Finding max salary for each department

In [16]:
connection.exec_driver_sql('SELECT '
                           '    d.department_name, '
                           '    MAX(s.amount) AS MAX_SALARY '
                           'FROM '
                           '    Salary AS s '
                           'JOIN '
                           '    Employees AS e ON s.employee_id = e.id '
                           'JOIN '
                           '    Department AS d ON e.department_id = d.id '
                           'JOIN '
                           '    Job_Title AS jt ON e.job_title_id = jt.id '
                           'GROUP BY '
                           '    d.department_name '
                           'ORDER BY '
                           '    MAX_SALARY DESC'
                           ).fetchall()

[('Service', 8453), ('IT', 7698), ('Finance', 6629)]

### Finding top 4 of employees with the highest salary

In [17]:
connection.exec_driver_sql('SELECT '
                           '    name, '
                           '    amount '
                           'FROM ('
                           '    SELECT '
                           '        e.name,'
                           '        s.amount'                           
                           '    FROM '
                           '        Salary AS s '
                           '    JOIN '
                           '        Employees AS e ON s.employee_id = e.id '
                           '    JOIN '
                           '        Department AS d ON e.department_id = d.id '
                           '    JOIN '
                           '        Job_Title AS jt ON e.job_title_id = jt.id'
                           '    ORDER BY s.amount DESC'
                           '    LIMIT 4) AS grouped_salaries '                           
                           ).fetchall()

[('Sam Smith', 8453),
 ('Jon Jones', 7713),
 ('Sherlock Holmes', 7698),
 ('Connor McGregor', 7644)]

### Closing the connection

In [18]:
connection.close()